# Head-to-head comparison of R, Python, Julia, and C/C++

System information (for reproducibility)

In [1]:
versioninfo()

Julia Version 1.7.3
Commit 742b9abb4d (2022-05-06 12:58 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin21.4.0)
  CPU: Intel(R) Core(TM) i7-6920HQ CPU @ 2.90GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-12.0.1 (ORCJIT, skylake)
Environment:
  JULIA_EDITOR = code
  JULIA_NUM_THREADS = 4


In [2]:
# load packages in environment
import Pkg
Pkg.activate("../../Docker")
Pkg.status()

  Activating project at `~/Documents/github.com/JSM-Julia-Short-Course-2022/Docker`


      Status `~/Documents/github.com/JSM-Julia-Short-Course-2022/Docker/Project.toml`
  [1520ce14] AbstractTrees v0.3.4
  [336ed68f] CSV v0.10.4
  [31c24e10] Distributions v0.25.66
  [38e38edf] GLM v1.8.0
  [28b8d3ca] GR v0.66.0
  [c27321d9] Glob v1.3.0
  [a15396b6] OnlineStats v1.5.13
  [ca7969ec] PlotlyLight v0.5.1
  [91a5bcdd] Plots v1.31.5
  [c3e4b0f8] Pluto v0.19.11
  [7f904dfe] PlutoUI v0.7.39
  [438e738f] PyCall v1.93.1
  [6f49c342] RCall v0.13.13
  [ce6b1742] RDatasets v0.7.7
  [2913bbd2] StatsBase v0.33.20
  [f3b207a7] StatsPlots v0.15.1
  [b8865327] UnicodePlots v3.0.4


## Types of computer languages

* **Compiled languages** (low-level languages): C/C++, Fortran, ... 
  - Directly compiled to machine code that is executed by CPU 
  - Pros: fast, memory efficient
  - Cons: longer development time, hard to debug

* **Interpreted language** (high-level languages): R, Matlab, Python, SAS IML, JavaScript, ... 
  - Interpreted by interpreter
  - Pros: fast prototyping
  - Cons: excruciatingly slow for loops

* Mixed (dynamic) languages: Matlab (JIT), R (`compiler` package), Julia, Cython, JAVA, ...
  - Pros and cons: between the compiled and interpreted languages

* Script languages: Linux shell scripts, Perl, ...
  - Extremely useful for some data preprocessing and manipulation

* Database languages: SQL, Hive (Hadoop).  
  - Data analysis *never* happens if we do not know how to retrieve data from databases  

## How high-level languages work?

- Typical execution of a high-level language such as R, Python, and Matlab.

<img src="./r-bytecode.png" align="center" width="400"/>

- To improve efficiency of interpreted languages such as R, Matlab and Python, conventional wisdom is to avoid loops as much as possible. Aka, **vectorize code**
> The only loop you are allowed to have is that for an iterative algorithm.

- When looping is unavoidable, need to code in C, C++, or Fortran. This creates the notorious **two language problem**  
Success stories: the popular `glmnet` package in R is coded in Fortran; `tidyverse` and `data.table` packages use a lot RCpp/C++.

<img src="./two_language_problem.jpg" align="center" width="600"/>

- High-level languages have made many efforts to bring themselves closer to the performance of low-level languages such as C, C++, or Fortran, with a variety levels of success.   
    - Matlab has employed JIT (just-in-time compilation) technology since 2002.  
    - Since R 3.4.0 (Apr 2017), the JIT bytecode compiler is enabled by default at its level 3.   
    - Cython is a compiler system based on Python.  

- Modern languages such as Julia tries to solve the two language problem. That is to achieve efficiency without vectorizing code.

- Julia execution.

<img src="./julia_compile.png" align="center" width="400"/>

<img src="./julia_introspect.png" align="center" width="400"/>

## Gibbs sampler example by Doug Bates

- Doug Bates (member of R-Core, author of popular R packages `Matrix`, `lme4`, `RcppEigen`, etc)
    
    > As some of you may know, I have had a (rather late) mid-life crisis and run off with another language called Julia.   
    >
    > -- <cite>Doug Bates (on the `knitr` Google Group)</cite>

- An example from Dr. Doug Bates's slides [Julia for R Programmers](http://www.stat.wisc.edu/~bates/JuliaForRProgrammers.pdf).

- The task is to create a Gibbs sampler for the density  
$$
f(x, y) = k x^2 exp(- x y^2 - y^2 + 2y - 4x), x > 0
$$
using the conditional distributions
$$
\begin{eqnarray*}
  X | Y &\sim& \Gamma \left( 3, \frac{1}{y^2 + 4} \right) \\
  Y | X &\sim& N \left(\frac{1}{1+x}, \frac{1}{2(1+x)} \right).
\end{eqnarray*}
$$

* R solution. The `RCall.jl` package allows us to execute R code without leaving the `Julia` environment. We first define an R function `Rgibbs()`.

In [3]:
using RCall

# show R information
R"""
sessionInfo()
"""

RObject{VecSxp}
R version 4.1.2 (2021-11-01)
Platform: x86_64-apple-darwin17.0 (64-bit)
Running under: macOS Big Sur 10.16

Matrix products: default
BLAS:   /Applications/Julia-1.7.app/Contents/Resources/julia/lib/julia/libblastrampoline.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/4.1/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

loaded via a namespace (and not attached):
[1] compiler_4.1.2


In [4]:
# define a function for Gibbs sampling

R"""
library(Matrix)

Rgibbs <- function(N, thin) {
  mat <- matrix(0, nrow=N, ncol=2)
  x <- y <- 0
  for (i in 1:N) {
    for (j in 1:thin) {
      x <- rgamma(1, 3, y * y + 4) # 3rd arg is rate
      y <- rnorm(1, 1 / (x + 1), 1 / sqrt(2 * (x + 1)))
    }
    mat[i,] <- c(x, y)
  }
  mat
}
"""

RObject{ClosSxp}
function (N, thin) 
{
    mat <- matrix(0, nrow = N, ncol = 2)
    x <- y <- 0
    for (i in 1:N) {
        for (j in 1:thin) {
            x <- rgamma(1, 3, y * y + 4)
            y <- rnorm(1, 1/(x + 1), 1/sqrt(2 * (x + 1)))
        }
        mat[i, ] <- c(x, y)
    }
    mat
}


To generate a sample of size 10,000 with a thinning of 500. How long does it take?

In [5]:
R"""
system.time(Rgibbs(10000, 500))
"""

RObject{RealSxp}
   user  system elapsed 
 22.740   1.751  24.717 


* This is a Julia function for the same Gibbs sampler:

In [6]:
using Distributions

function jgibbs(N, thin)
    mat = zeros(N, 2)
    x = y = 0.0
    for i in 1:N
        for j in 1:thin
            x = rand(Gamma(3, 1 / (y * y + 4)))
            y = rand(Normal(1 / (x + 1), 1 / sqrt(2(x + 1))))
        end
        mat[i, 1] = x
        mat[i, 2] = y
    end
    mat
end

jgibbs (generic function with 1 method)

Generate the same number of samples. How long does it take?

In [7]:
jgibbs(100, 5); # warm-up
@elapsed jgibbs(10000, 500)

0.279728267

We see 40-80 fold speed up of `Julia` over `R` on this example, **with similar coding effort**!

## Comparing C, R, Python, and Julia

To better understand how these languages work, we consider a simple task: summing a vector.

Let's first generate data: 1 million double precision numbers from uniform [0, 1].

In [8]:
using Random # standard library
Random.seed!(123) # seed
x = rand(1_000_000) # 1 million random numbers in [0, 1)
sum(x)

500220.6882968192

In this class, we use package `BenchmarkTools.jl` for robust benchmarking. It's analog of `microbenchmark` or `bench` package in R.

In [9]:
using BenchmarkTools

### C

We would use the low-level C code as the baseline for copmarison. In Julia, we can easily run compiled C code using the `ccall` function.

In [10]:
using Libdl

C_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
"""

const Clib = tempname()   # make a temporary file

# compile to a shared library by piping C_code to gcc
# (works only if you have gcc installed):

open(`gcc -std=c99 -fPIC -O3 -msse3 -xc -shared -o $(Clib * "." * Libdl.dlext) -`, "w") do f
    print(f, C_code) 
end

# define a Julia function that calls the C function:
c_sum(X::Array{Float64}) = ccall(("c_sum", Clib), Float64, (Csize_t, Ptr{Float64}), length(X), X)

c_sum (generic function with 1 method)

In [11]:
# make sure it gives same answer
c_sum(x)

500220.6882968189

In [12]:
bm = @benchmark c_sum($x)

BenchmarkTools.Trial: 4238 samples with 1 evaluation.
 Range (min … max):  1.124 ms …  1.761 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.161 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.176 ms ± 63.037 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   █▆   ▁▂                                                    
  ████▄▄███▅▄▄▃▃▃▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▂ ▃
  1.12 ms        Histogram: frequency by time        1.46 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [13]:
using Statistics # standard library
benchmark_result = Dict() # a dictionary to store median runtime (in milliseconds)

# store median runtime (in milliseconds)
benchmark_result["C"] = median(bm.times) / 1e6

1.1608855

### R, buildin sum

Next we compare to the build in `sum` function in R, which is implemented using C.

In [14]:
using RCall

R"""
library(bench)
y <- $x
rbm_builtin <- bench::mark(sum(y))
"""

RObject{VecSxp}
# A tibble: 1 × 13
  expression      min median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc total_time
  <bch:expr> <bch:tm> <bch:>     <dbl> <bch:byt>    <dbl> <int> <dbl>   <bch:tm>
1 sum(y)       2.38ms 2.46ms      402.        0B        0   202     0      503ms
# … with 4 more variables: result <list>, memory <list>, time <list>, gc <list>


In [15]:
# store median runtime (in milliseconds)
@rget rbm_builtin # dataframe
benchmark_result["R builtin"] = median(rbm_builtin[!, :median]) * 1000

2.4625275000014213

### R, handwritten loop

Handwritten loop in R is much slower.

In [16]:
using RCall

R"""
sum_r <- function(x) {
  s <- 0
  for (xi in x) {
    s <- s + xi
  }
  s
}
library(bench)
y <- $x
rbm_loop <- bench::mark(sum_r(y))
"""

RObject{VecSxp}
# A tibble: 1 × 13
  expression      min median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc total_time
  <bch:expr> <bch:tm> <bch:>     <dbl> <bch:byt>    <dbl> <int> <dbl>   <bch:tm>
1 sum_r(y)     12.7ms   13ms      76.1    12.7KB        0    39     0      513ms
# … with 4 more variables: result <list>, memory <list>, time <list>, gc <list>


In [17]:
# store median runtime (in milliseconds)
@rget rbm_loop # dataframe
benchmark_result["R loop"] = median(rbm_loop[!, :median]) * 1000

13.011620000000335

### R, Rcpp

Rcpp package is the easiest way to incorporate C++ code in R.

In [18]:
R"""
library(Rcpp)

cppFunction('double rcpp_sum(NumericVector x) {
  int n = x.size();
  double total = 0;
  for(int i = 0; i < n; ++i) {
    total += x[i];
  }
  return total;
}')
rcpp_sum
"""

RObject{ClosSxp}
function (x) 
.Call(<pointer: 0x11042c850>, x)


In [19]:
R"""
rbm_rcpp <- bench::mark(rcpp_sum(y))
"""

RObject{VecSxp}
# A tibble: 1 × 13
  expression      min median `itr/sec` mem_alloc `gc/sec` n_itr  n_gc total_time
  <bch:expr>  <bch:t> <bch:>     <dbl> <bch:byt>    <dbl> <int> <dbl>   <bch:tm>
1 rcpp_sum(y)  1.17ms 1.21ms      814.    2.49KB        0   407     0      500ms
# … with 4 more variables: result <list>, memory <list>, time <list>, gc <list>


In [20]:
# store median runtime (in milliseconds)
@rget rbm_rcpp # dataframe
benchmark_result["R Rcpp"] = median(rbm_rcpp[!, :median]) * 1000

1.2082829999986444

### Python, builtin sum

Built in function `sum` in Python.

In [21]:
using PyCall
PyCall.pyversion

v"3.7.6"

In [22]:
# get the Python built-in "sum" function:
pysum = pybuiltin("sum")
bm = @benchmark $pysum($x)

BenchmarkTools.Trial: 67 samples with 1 evaluation.
 Range (min … max):  71.462 ms … 99.945 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     74.757 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   75.408 ms ±  3.503 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

              ▄█▂▄▂   ▂    ▂                                   
  ▄▁▁▆▁▄▁▁▁▁▄▁█████▄█▆███▆▄█▁▄▁▆▁▆▄▁▄▄▁▆▁▁▁▁▄▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▄ ▁
  71.5 ms         Histogram: frequency by time        81.7 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [23]:
# store median runtime (in miliseconds)
benchmark_result["Python builtin"] = median(bm.times) / 1e6

74.757128

### Python, handwritten loop

In [24]:
using PyCall

py"""
def py_sum(A):
    s = 0.0
    for a in A:
        s += a
    return s
"""

sum_py = py"py_sum"

bm = @benchmark $sum_py($x)

BenchmarkTools.Trial: 53 samples with 1 evaluation.
 Range (min … max):  93.538 ms … 104.348 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     95.496 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   95.787 ms ±   2.257 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅▂           ▂   ▅  ▅                                        
  █████▅▅▅█▅▅▅▅▁█████▁▁█▁▅▁▁▁▅▅▁▅▁▅▁▁▅▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▁▁▁▅▁▁▁▅ ▁
  93.5 ms         Histogram: frequency by time          102 ms <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [25]:
# store median runtime (in miliseconds)
benchmark_result["Python loop"] = median(bm.times) / 1e6

95.496408

### Python, numpy

Numpy is the high-performance scientific computing library for Python.

In [26]:
# bring in sum function from Numpy 
numpy_sum = pyimport("numpy")."sum"

PyObject <function sum at 0x7fc410068b90>

In [27]:
bm = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  378.482 μs …  1.160 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     412.020 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   434.387 μs ± 64.678 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▆▇▇█▇▆▅▆▅▅▄▄▄▄▄▄▃▃▂▁▁▁▁                                     ▂
  ██████████████████████████████████▇█▇▆▆▆▇▆▆▇▆▆▅▇▅▆▇▅▆▇▅▅▅▄▅▄ █
  378 μs        Histogram: log(frequency) by time       726 μs <

 Memory estimate: 240 bytes, allocs estimate: 6.

In [28]:
# store median runtime (in miliseconds)
benchmark_result["Python numpy"] = median(bm.times) / 1e6

0.41202

Numpy performance is on a par with Julia build-in sum function. Both are about 3 times faster than C, possibly because of usage of [SIMD](https://en.wikipedia.org/wiki/SIMD).

### Julia, builtin sum

`@time`, `@elapsed`, `@allocated` macros in Julia report run times and memory allocation.

In [29]:
@time sum(x) # no compilation time after first run

  0.000466 seconds (1 allocation: 16 bytes)


500220.6882968192

For more robust benchmarking, we use BenchmarkTools.jl package.

In [30]:
bm = @benchmark sum($x)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  264.284 μs … 701.111 μs  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     295.739 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   301.928 μs ±  27.926 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

      ▁▄▅▆▇███▇▆▄▄▄▂▂▁                                           
  ▁▂▄▆█████████████████▇▆▅▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  264 μs           Histogram: frequency by time          412 μs <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [31]:
benchmark_result["Julia builtin"] = median(bm.times) / 1e6

0.295739

### Julia, handwritten loop

Let's also write a loop and benchmark.

In [32]:
function jl_sum(A)
    s = zero(eltype(A))
    for a in A
        s += a
    end
    s
end

bm = @benchmark jl_sum($x)

BenchmarkTools.Trial: 4177 samples with 1 evaluation.
 Range (min … max):  1.133 ms …  1.987 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.176 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.193 ms ± 65.523 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▄▆▂▁ ▃█▄▃▁                                                 
  ▇████▆█████▆▄▄▂▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  1.13 ms        Histogram: frequency by time        1.45 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [33]:
benchmark_result["Julia loop"] = median(bm.times) / 1e6

1.175818

**Exercise**: annotate the loop by [@simd](https://docs.julialang.org/en/v1/manual/performance-tips/#man-performance-annotations-1) and benchmark again. 

### Summary

In [34]:
sort(collect(benchmark_result), by = x -> x[2])

9-element Vector{Pair{Any, Any}}:
  "Julia builtin" => 0.295739
   "Python numpy" => 0.41202
              "C" => 1.1608855
     "Julia loop" => 1.175818
         "R Rcpp" => 1.2082829999986444
      "R builtin" => 2.4625275000014213
         "R loop" => 13.011620000000335
 "Python builtin" => 74.757128
    "Python loop" => 95.496408

* `C`, `R builtin` are the baseline C performance (gold standard).

* `Python builtin` and `Python loop` are 80-100 fold slower than C because the loop is interpreted.

* `R loop` is about 30 folder slower than C and indicates the performance of JIT bytecode generated by its compiler package (turned on by default since R v3.4.0 (Apr 2017)). 

* `Julia loop` is close to C performance, because Julia code is JIT compiled.

* `Julia builtin` and `Python numpy` are 3-4 folder faster than C because of SIMD.

## Take home message for computational scientists

- High-level language (R, Python, Matlab) programmers should be familiar with existing high-performance packages.  Don't reinvent wheels.  
    - R: RcppEigen, tidyverse, ...   
    - Python: numpy, scipy, ...  

- In most research projects, looping is unavoidable. Then we need to use a low-level language.  
    - R: Rcpp, ...  
    - Python: Cython, ...  
    
- In this course, we use Julia, which seems to circumvent the two language problem. So we can spend more time on algorithms, not on juggling $\ge 2$ languages.  